In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from dataset.dataset import Shipwreck
from net.ModifiedGenerator import *
import os
from torchvision.utils import save_image
from net.Ushape_Trans import *
from net.utils import *

In [2]:
batch_size = 1
lr = 0.0002
num_epochs = 1

In [3]:
shipwreck_train = Shipwreck('_','dataset/d1/sonar')
trainloader = torch.utils.data.DataLoader(shipwreck_train, batch_size=batch_size, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

orginalGenerator = Generator().to(device)
originalDiscriminator = Discriminator().to(device)

/home/hassan/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
orginalGenerator.load_state_dict(torch.load("./saved_models/G/generator_795.pth", map_location=device))
orginalGenerator.eval()
modifiedGenerator = modifiedGenerator(orginalGenerator).to(device)
modifiedGenerator.train()

modifiedGenerator(
  (model): Generator(
    (linear_encoding): Linear(in_features=384, out_features=512, bias=True)
    (position_encoding): LearnedPositionalEncoding()
    (pe_dropout): Dropout(p=0.0, inplace=False)
    (transformer): TransformerModel(
      (net): IntermediateSequential(
        (0): Residual(
          (fn): PreNormDrop(
            (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (fn): SelfAttention(
              (qkv): Linear(in_features=512, out_features=1536, bias=False)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=512, out_features=512, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
          )
        )
        (1): Residual(
          (fn): PreNorm(
            (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (fn): FeedForward(
              (net): Sequential(
         

In [5]:
optimizer_Generator = torch.optim.Adam(modifiedGenerator.parameters(), lr=lr,  betas=(0.5, 0.999))

In [6]:
from loss.LAB import lab_Loss
from loss.LCH import lch_Loss
import pytorch_ssim


MSE = nn.MSELoss(size_average=False).to(device)
SSIM = pytorch_ssim.SSIM().to(device)
L_vgg = VGG19_PercepLoss().to(device)
L_lab=lab_Loss().to(device)
L_lch=lch_Loss().to(device)

/home/hassan/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/home/hassan/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hassan/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel=0.1
lambda_lab=0.001
lambda_lch=1
lambda_con = 100
lambda_ssim=100

In [8]:
for epoch in range(num_epochs):
    for i, img in enumerate(trainloader):
        optimizer_Generator.zero_grad()

        img = img.to(device)
        rgb_img,_ = shipwreck_train.split_tensor(img)
        rgb_img = rgb_img.to(device)
        
        groundTruth = orginalGenerator(rgb_img)

        fake = modifiedGenerator(img)
        print(rgb_img.shape)
        save_image(rgb_img,"rgb_img.png")
        save_image(groundTruth[3].data, 'gt.png', normalize=True)
        save_image(fake[3].data, 'fake.png', normalize=True)
        exit(0)

        groundTruth = groundTruth[3].data
        fake = fake[3].data

        loss_pixel = MSE(fake, groundTruth)
        loss_ssim = 1-SSIM(fake, groundTruth)
        loss_vgg = L_vgg(fake, groundTruth)
        loss_lab = L_lab(fake, groundTruth)
        loss_lch = L_lch(fake, groundTruth)

        loss_G = (lambda_pixel * loss_pixel+lambda_ssim*loss_ssim+lambda_con*loss_vgg+lambda_lab*loss_lab+lambda_lch*loss_lch)

        loss_G.backward()
        optimizer_Generator.step()  

torch.Size([1, 3, 256, 256])


AttributeError: 'list' object has no attribute 'size'

: 